In [3]:
!pip list | grep scikit-learn

scikit-learn                  1.2.1
scikit-learn-intelex          20230228.214242


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ubuntu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
df['y_pred'] = y_pred
df['y_pred'].std()

5.281404481465004

In [11]:
year = 2022
month = 2
output_file = 'output.parquet'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = df [['ride_id', 'y_pred']]
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
import os

def get_parquet_file_size_in_mb(parquet_file_path):
  """Returns the size of the parquet file in megabytes.

  Args:
    parquet_file_path: The path to the parquet file.

  Returns:
    The size of the parquet file in megabytes.
  """

  file_size_bytes = os.stat(parquet_file_path).st_size
  file_size_mb = file_size_bytes / (1024 * 1024)
  return file_size_mb

if __name__ == "__main__":
  parquet_file_path = output_file
  file_size_mb = get_parquet_file_size_in_mb(parquet_file_path)
  print(f"The size of the parquet file is {file_size_mb} MB.")

The size of the parquet file is 57.215529441833496 MB.


In [13]:
!jupyter nbconvert --to python starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to python
[NbConvertApp] Writing 1869 bytes to starter.py
